In [1]:
import json
import requests
import ipywidgets as widgets
from IPython.display import clear_output, Markdown, display
from time import time

endpoint = "http://localhost:9200"
username = "admin"
password = "admin"

def search_prefix(keywords):
    if not keywords:
        return None
    keywords = keywords.split(' ')
    if len(keywords) > 1:
        prefix = keywords[-1]
        phrase = ' '.join(keywords[:-1])
        query = {
            'query': {
                'bool': {
                    'must': [
                        {
                            'prefix': {
                                'title': prefix
                            }
                        },
                        {
                            'match': {
                                'title': {
                                    'query': phrase,
                                    'minimum_should_match': '100%'
                                }
                            }
                        }
                    ]
                }
            }
        }
    else:
        keywords = ' '.join(keywords)
        query = {
            'query': {
                'prefix': {
                    'title': keywords
                }
            }
        }

    query = json.dumps(query)

    url = endpoint + '/nutch/_search'
    r = requests.post(url, 
                      auth=(username, password), 
                      data=query, 
                      headers={'Content-type': 'application/json'})
    return r.json()

def bold(result, keywords):
    """Bold keywords in the result"""
    keywords = keywords.strip().split(' ')
    for keyword in keywords:
        result = result.replace(keyword, f"**{keyword}**")
        keyword = keyword.title()
        result = result.replace(keyword, f"**{keyword}**")
    return result

def truncate_text(content, max_lines=6):
    """Truncate content to a maximum number of lines"""
    lines = content.split('\n')
    truncated = '\n'.join(lines[:max_lines])
    return truncated + '...' if len(lines) > max_lines else truncated

def format_result(res, keywords):
    """Format the result for display"""
    title = bold(res.get('title', 'No Title'), keywords)
    content = bold(truncate_text(res.get('content', 'No Content')), keywords)
    doc_id = res.get('id', '#')
    
    # Use the ID as a hyperlink for the title
    return f"### [{title}]({doc_id})\n\n{content}"

def printmd(string):
    """Wrapper function to display markdown"""
    display(Markdown(string))

def text_change(change):
    """Widget event when the value changes"""
    global output
    with output:
        clear_output()
        t0 = time()
        results = search_prefix(change['new'])
        if results:
            hits = results['hits']['hits']
            print_text = []
            for res in hits[:]:
                res = res['_source']
                print_text.append(format_result(res, change['new']))
            printmd('<br><br>'.join(print_text))
            if hits:
                print(f'\n{round(time() - t0, 4)} seconds')

box = widgets.Text(
         value='',
         placeholder='Type something',
         description='Query:',
         disabled=False
     )
output = widgets.Output()
display(box, output)
box.observe(text_change, names='value')


Text(value='', description='Query:', placeholder='Type something')

Output()